# Gestion de al informacion en una BD

## Creación de Tablas

Acciones:

- CREATE
- ALTER
- DROP

### pgadmin

Para mostrar el proceso desde cero, eliminaremos por completo la BD *transporte*

    drop database transporte_publico;

Sobre la parte izquierda, click, create database, solo le damos el nombre, y dejamos como propietario al usuario *postgres*

![](https://i.imgur.com/0EYRSoU.png)

Fijate en la pestaña SQL, esta el comando que si quisieramos podriamos ejecutar en la consola.

![](https://i.imgur.com/C1XpH6d.png)

Le damos que si, y ya tenemos un BD vacia.

![](https://i.imgur.com/LuzGQho.png)

Toda esa configuracion como los triggers, extension, los veremos mas adelante. De manera predeterminada, se creo un esquema que es el publico. 

![](https://i.imgur.com/yIQ14hj.png)

Y donde dice *tables* crearemos nuestra primera tabla:

![](https://i.imgur.com/UHC8jGg.png)

- serial es autoincremental
- character varying seria el equivalente a varchar en otros motores. 

####  Llave Primaria

ir a la pestaña *constraints* en la subpestaña *Primary key*, y el nombre que le vamos a colocar va a ser una convencion: *el nombre de la tabla* seguido de *pkey* 

Y por ahora no modificaremos los parametros avanzados. Solo inspeccionaremos la pestaña SQL, comprobamos todo este bien, y la creamos(en nuestro caso modificamos):

![](https://i.imgur.com/yUaf0L1.png)

#### Script de insercion:

Desde pgadmin muy facil: INSERT script

![](https://i.imgur.com/YTTjszo.png)

Como id es un campo que se incrementa automaticamente con cada INSERT, una buena practica es quitarlo del Script de insercion.

Para ver el formato de inserccion de fechas, lo podemos verificar en el archivo de configuracion, o mejor aun, hacerle una consulta a la BD de la fecha actual

    SELECT current_date;

Y listo, lo tenemos:

    INSERT INTO public.pasajero(
	name, home_address, birthdate)
	VALUES ('Primer Pasajero', 'alguna direccion', '2023-04-07');

## PARTICIONES

![](https://i.imgur.com/rZBOppR.png)

En ocasiones llega haber demasiada informacion en una tabla, y para consultarla. se tendra que recorrer toda la informacion para encontrar lo que se esta buscando. 

Por ejemplo, la tabla viajes, va a tener alrededor de un millon de filas, y queremos saber los viajes entre tal fecha y tal otra, ¿Que se puede hacer?

Se podria hacer una separacion fisica de los datos, pero que conserve la misma estructura logica, es decir guardar diferentes partes de la tabla, en diferentes partes de disco, incluso en otros discos. Y como tiene la misma estructura logica, podriamos por ejemplo usar el mismo *SELECT*.

Y creamos, por ejemplo, una particion para enero, otra para febrero, etc, etc. Y si le damos una consulta de un mes en particular, no buscaria en toda la tabla, sino en la correspondiente a ese mes. 

### Practica

Crearemos una nueva tabla: *bitacora_viajes* con tres campos:

- id
- id_viaje
- fecha

Siendo fecha el campo por el cual vamos a particionar la tabla. 

Hacerlo desde *pgadmin*. 

![](https://i.imgur.com/rmLTnNs.png)

❌ Fijate no le colocamos primary key ni not null, para que cuando vayamos a la tab de constrains, no salga ninguna. 

👉 Importante aclarar que en tablas particionadas, no se usan llaves primarias porque no es posible

- En la pestaña general le indicamos que es una tabla particionada

Hasta ahora esto es lo que hemos hecho si revisamos la SQL tab, pero todavia no esta listo:

    CREATE TABLE public.bitacora_viajes
    (
        id serial,
        id_viaje integer,
        fecha date
    ) PARTITION BY RANGE (fecha);

    ALTER TABLE IF EXISTS public.bitacora_viajes
        OWNER to postgres;

Pero ojo todavia no se ha agregado una tabla particionanada, por tanto en este punto ni siquiera podremos insertar datos, ni nada. Igualmente la creamos. Y nota el icono es diferente:

![](https://i.imgur.com/I0LVaX8.png)

Si en este punto, intetaramos hacer una insercion, mira que el error que saldria:

    ERROR:  no partition of relation "bitacora_viajes" found for row
    DETAIL:  Partition key of the failing row contains (fecha) = (2010-05-05).
    SQL state: 23514

Entonces creemos una particion, hagamolo como una consulta SQL:

    CREATE TABLE bitacora_viajes201005 PARTITION OF bitacora_viajes
    FOR VALUES FROM ('2010-05-01') TO ('2010-05-31');

- La particion en realidad es una tabla, por lo que es importante crear tambien un nombre descriptivo: que es del mes de mayo.

Ahora si intentenomos nuevamente insertar una fila dentro de ese rango:

    INSERT INTO public.bitacora_viajes(
	id_viaje, fecha)
	VALUES (1, '2010-05-05');


### Ejercicio. 

Crear una particion para el año 2009, e insertar un dato:

    CREATE TABLE bitacora_viajes2009 PARTITION OF bitacora_viajes
    FOR VALUES FROM ('2009-01-01') TO ('2009-12-31');

Y el valor:

    INSERT INTO public.bitacora_viajes(
	    id_viaje, fecha)
	    VALUES (2, '2009-12-30');


Y hagamos un select general, para verificar hay dos datos en la tabla:

    SELECT * FROM bitacora_viajes;

Explorando un poco mas el arbol de la BD:

![](https://i.imgur.com/qWAwIqn.png)

No se usaran partciones en el proyecto. Asi que si quieres borrala. 

## Creacion de Roles

### Que puede hacer un ROL

- Crear y eliminar
- Asignar Atributos
- Agrupar con otros roles
- Roles Predeterminados

El usuario *postgres* tiene todo el poder de borrar toda la BD, asi que lo ideal es crear un usuario con los niveles de seguridad indicados y usar este usuario.

Los roles son independientes a las BD.

#### Comando CREATE ROL

Si no estamos seguros que caracteristicas existen, entonces:

    \h CREATE ROL

![](https://i.imgur.com/0uYL1T0.png)

La consulta anterior es muy importante la revises, pues es bastante intuitiva. 

#### Crear un rol que tenga la capacidad de hacer login y asignarle una contraseña

    CREATE ROLE usuario_consulta;

Recuerda tambien hubieras podido usar *CREATE USER* o *CREATE USUARIO* son alias de *CREATE ROL* apartir de la version 9.3.

Para observar todos los roles creados con sus atributos:

    \dg

![](https://i.imgur.com/SwCMGqe.png)

En este caso. como simplemente se creo el Rol, no hay ningun atributo asociado a este. Entonces modificamos el ROL:

    ALTER ROLE usuario_consulta WITH LOGIN;

En este punto, aclaro que el funcionameinto de PSQL es diferente en Linux y Windows, por lo que algunos aportes del instructor no son validos para mi SO.

Si ahora quiero moverme a otro rol:

    set role usuario_consulta;

Y para verificar:

    set role usuario_consulta;

![](https://i.imgur.com/soIaI86.png)


Recordemos el role *usuario_consulta* se creo sin un password, y eso podria ser un problema cuando intentemos ingresar a pgadmin.

Entonces mejor asignemole un password en este momento:

    ALTER ROLE usuario_consulta WITH PASSWORD 'etc1234';

Si en este punto, intentamos hacer una consulta de la tabla en la BD de tranporte, pues nos dara un error de acceso:

    SELECT * FROM pasajero;

Dara:

    permission denied for table pasajero

Porque los permisos son insuficientes, cuando creemos el usuario desde pgadmin, le daremos los permisos indicados.

#### Registrando el usuario en pgadmin:

Vamos a Object --> Register --> Server.

- Le colocamos un nombre cualquiera: usuario-cons
- hostname: localhost
- user: usuario_consulta

#### Borrando el role

    DROP ROLE usuario_consulta ;

Verifiamos con \dg

### Creando el usuario con Pgadmin

Desde el arbol asociado a la conexion con el usuario *postgres*, verificamos los roles creados:

- postgres
- curso

Se pueden distinguir un usuario de un grupo de permisos, porque los permisos aparecen como grupos, en este caso *pg_read_all_data*

![](https://i.imgur.com/dDCFE9t.png)

Vamos hacer *click derecho sobre LOGIN/Group Roles* --> Crear y abrira una nueva ventana:

- General: name: usuario_consulta
- Definition: Password: etc1234

![](https://i.imgur.com/VqP0Dqi.png)

En Privilegios, solo le damos que pueda logear, porque al activar que herede los privilegios del padre, y al ser el padre, postgress, podra leer, escribir, sin necesidad de darle permisos de superusuario. 

En Parametros, podriamos limitar el uso de memoria, entre otros,  si quisieramos.

Y asi lo creo:

![](https://i.imgur.com/BrnTjL6.png)

Y como vemos,  no puede crear BD, ni Roles, ni replicaciones. 

Este *usuario_consulta* lo podrias compartir con tu equipo para que consulten la BD CRUD, pero no puedan modificar su estructura.

Pero aun falta algo mas. Si a este punto intentas hacer un SELECT:

    SELECT * FROM pasajero;

Pues te seguira dando un error de acceso. Entonces modificaremos los permisos para el *usuario_consulta*. Sobre el arbol de todas las tablas, si asi lo queremos, le damos la opcion de *Grant Wizard*

![](https://i.imgur.com/4SpXGn7.png)

Seleccionamos todas las tablas, si asi lo deseamos

![](https://i.imgur.com/C00BT7c.png)

Le damos siguiente, y a continuacion *add row*. Observamos se le van a dar permisos de INSERT, SELECT and UPDATE por parte del superusuario *curso* a nuestro *usuario_consulta*. 

![](https://i.imgur.com/zQzaYA3.png)

Si consultamos nuevamente sobre una tabla pasajero, hay unos permisos especiales para nuestro usuario

![](https://i.imgur.com/yhocxZZ.png)

Si nuevamente ejecutamos un SELECT, esta vez sera exitoso 😇:

![](https://i.imgur.com/G6b30zY.png)


## Llaves foráneas

Las tablas tienen llaves primarias, y las llaves foraneas, se refiere a la relacion que existe entre las tablas, y corresponde a la (C)oncistency de nuestro ACID standart. 

### Estructura Llaves foraneas

![](https://i.imgur.com/oQM5d5N.png)

Las acciones se refiere al caso de una modificacion en la tabla de origen

### Creanda las tablas basicas

Como *tren* y *estacion*. Aqui para *estacion* 

![](https://i.imgur.com/lb2r7a2.png)

![](https://i.imgur.com/aexz6cl.png)


### Creando tablas de relaciones

Para la tabla *trayecto*:

![](https://i.imgur.com/sYduq5d.png)

Y creamos las llaves foraneas, illendo a la pestaña Constrains->Foreing key:

- la convencion es nombre de la tabla origen(trayecto) y a continuacion la tabla destino: *trayecto_estacion_fkey*

- En la pestaña de *Definition* hay caracteristicas muy especiales tales como llave foranea validada al instante, o sea validada despues, o no sea validada al momento de un backup

![](https://i.imgur.com/ktmS7kT.png)

La pestaña accion tambien es muy importante, porque se le dice al motor de la BD que hacer en caso de que ocurra un cambio(ACTUALIZACION O BORRADO)

Si ocurre un cambio en la tabla principal que en este caso es *estacion*, ¿que hacer con la tabla trayecto? 

- NO ACTION
- RESTRICT
- CASCADE
- SET NULL
- SET DEFAULT

Para el ejercicio lo dejaremos NO ACTION


- Creando la relacion de trayecto con tren:

![](https://i.imgur.com/jOSJNFz.png)

#### Creando la tabla viajes

![](https://i.imgur.com/mqWpr2z.png)

![](https://i.imgur.com/E56DqfP.png)